# Demonstration of some different ways up populating and updating the tables

In [1]:
# Import modules that will be used
import populatedatabase
import findsta
import numpy as np

%pylab
#%matplotlib inline

# Define database file location
database='/Users/kallstadt/LSseis/landslideDatabase/lsseis.db'
# Define relative file path for sac files
path = '/Users/kallstadt/LSseis/landslideDatabase'

%load_ext autoreload
%autoreload 2

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


### Populate or repopulate station and sta_nearby tables for a range of event ids
Note, this can take a few minutes, especially if maxradius is large in an area where station spacing is dense.

In [2]:
# Run just for one event id out to 200km.
populatedatabase.initial_populate(120, minradius=0., maxradius=200., 
                 clients=['IRIS'], database=database)

1365
added 114 entries to stations table, 1251 already were there
added 247 entries to sta_nearby table, updated 1118 entries, 0 left as is, 0 not added because of error


In [5]:
# Do range of ids, this will do event 10 through 12 out to 200 km (last number is not included in arange
event_ids = np.arange(10, 13)
populatedatabase.initial_populate(event_ids, minradius=0., maxradius=200., 
                 clients=['IRIS', 'NCEDC'], database=database)

838
added 0 entries to stations table, 838 already were there
added 0 entries to sta_nearby table, updated 838 entries, 0 left as is, 0 not added because of error
304
added 0 entries to stations table, 304 already were there
added 0 entries to sta_nearby table, updated 304 entries, 0 left as is, 0 not added because of error
84
added 6 entries to stations table, 78 already were there
added 6 entries to sta_nearby table, updated 78 entries, 0 left as is, 0 not added because of error
6
added 0 entries to stations table, 6 already were there
added 0 entries to sta_nearby table, updated 6 entries, 0 left as is, 0 not added because of error
1084
added 34 entries to stations table, 1050 already were there
added 748 entries to sta_nearby table, updated 336 entries, 0 left as is, 0 not added because of error
363
added 0 entries to stations table, 363 already were there
added 0 entries to sta_nearby table, updated 363 entries, 0 left as is, 0 not added because of error


### Recalculate source to station distances, azimuth, and backazimuth for range of event ids
This will overwrite existing values using the current lat and lons for stations and source centroids

In [6]:
event_ids = np.arange(10,13)
populatedatabase.recalculate_distances(event_ids, database=database)

### Browse seismic data, populate database with information about what stations picked up what events in which frequency bands
#### You may not be able to do this step inside the notebook, it probably needs to be done in ipython with pylab initiated.
This will fill in detect_HF and detect_LP in sta_nearby, no measurements will be made yet, that will be done in make_measurements (in progress)
This will download data from IRIS each time it is run, so try not to run the same event over and over again
It will also load in sac data that is not available at IRIS if the paths in the database are relative and the full path is defined correctly.

You will do this interactively so **it will probably not work in this notebook** so here are some instructions:
1. The program will load in all the seismic it can find within the initial radius limits you define from local sac files as well as IRIS and NCEDC, it gets this information from the database so if the station and sta_nearby tables haven't been populated out to maxradius, it will populate them before proceeding.
2. The program will display these data in an interactive plot first for high frequencies, then for long periods, delete any traces that don't show any sign of the signal above the noise level or otherwise look useless (too noisy etc., many horizontal channels will be noisy or useless in the long period band), then press Q to exit, answer the prompts about whether you thought you had reached the furthest limit in this batch or not, then continue, loading data further and further out until you can't see the signal anymore.
3. For high frequencies, look one or two hundred km past where you think the last detection was, often there are stations further out that happened to have low noise that will have picked them up.
4. For long periods, the main objective is to see whether there are any noticeable long period signals or not. If there are, and it was a big event, you will likely be able to see the signals for thousands of km, so only go out to 500 km, when you reach that point, just enter >500 for the maximum distance observed.
5. In some cases, especially for lahars and other long events, the long periods may not be obvious except by looking at how waves are correlated between events. Sometimes it takes deleting the horizontal channels (since they have more noise) to be able to see this.
6. For long duration high frequency events like debris flows at volcanoes, try adding a larger buffer and zooming out to look at the entire time period, these often are hard to see without looking at long periods of time because they are gradual increases in amplitude.


In [ ]:
buffer_sec = 200. # The number of seconds prior to and after the event time to download, helps make signal easier to see
minradius = 0. # Minimum distance from source to search for stations in km
maxradius = 200. # Maximum distance from source to search for stations in km, this will be gradually increased by intincrkm until you no longer can see the signal on any stations
intincrkm = 100. # Interval to increase maxradius each time, do not make too large or it will bog down your computer
maxtraces = 15 # Maximum number of traces to display at a time

# Leave all other things as is to remain consistent between events

# This can take a while to load the data depending on how many stations there are and the duration of the event.

event_id = 11
populatedatabase.review_event(event_id, buffer_sec=buffer_sec, minradius=minradius, maxradius=maxradius,
             intincrkm=intincrkm,maxreachedHF=False, maxreachedLP=False, 
             HFlims=(1., 5.), LPlims=(20., 60.), LPoutput='DISP', maxtraces=15,
             database=database, path=path)

#### If you got to a certain distance and either made a mistake or crash the program, you don't have to start over
Just run review_event again, updating minradius and maxradius to correspond to the interval you were on when it crashed. If you had already reached the maximum limit for HF or LP, set maxreachedHF or maxreachedLP to True

### It might be a good idea to spot check the database using Views to make sure things you expect to happen are happening

### Next step is to go through the data from stations you marked as detections in both frequency bands and make seismic measurements on this data

#### First, the high frequency band
Notes:
* This also needs to be run in ipython with %pylab enabled for the Interactive Plotting to work
* First, the raw data will appear, look at all the data and make sure none of it is clipped. Delete any clipped channels
* Then press C to do the station corrections
* Then make an amplitude pick on each station (unless the signal looks wonky), being careful to not confuse the taper that was applied to the data with the start and end time of the signal. The start and end time of the amplitude picks is what will be defined as the start and end time of the signal on this station and will be used to calculate duration
* Add a larger buffer than usual so the taper doesn't interfere with selecting the start and end times. A few hundred seconds at least.
* If your computer gets sluggish, be sure to leave the mouse at each pick location until you see text printed, if you move it before then, it may not get the start time right.
* After making all your picks, press Q and calculations will be made using your picks and the database will be populated with this info
* Would be good to check the database using a View to make sure this data is being populated as expected
* If you exit out without pressing Q, the picks won't be saved.
* If you mess up a pick, make the pick again, the newer one will overwrite the older one.
* For events that were recording on many stations, it may overwhelm your computer to load them in all at once, so do several iterations, stepping up the radius range

In [ ]:
# To run the high frequency measurement analysis
# Do not change HFlims, HFoutput, LPlims or LPoutput from their default 
#  values in order to stay consistent across all events
populatedatabase.make_measurementsHF(60, buffer_sec=300., HFlims=(1., 5.), HFoutput='VEL',
                        minradius=0., maxradius=None, maxtraces=15, path='/Users/kallstadt/LSseis/landslideDatabase',
                        database='/Users/kallstadt/LSseis/landslideDatabase/lsseis.db')

# To run the long period measurement analysis
make_measurementsLP(11, buffer_sec=None, LPlims=(20., 60.), LPoutput='DISP',
                        minradius=0., maxradius=200., maxtraces=15, path='/Users/kallstadt/LSseis/landslideDatabase',
                        database='/Users/kallstadt/LSseis/landslideDatabase/lsseis.db')
make_measurementsLP(11, buffer_sec=None, LPlims=(20., 60.), LPoutput='DISP',
                        minradius=200., maxradius=400., maxtraces=15, path='/Users/kallstadt/LSseis/landslideDatabase',
                        database='/Users/kallstadt/LSseis/landslideDatabase/lsseis.db')